In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df1 = pd.read_csv("pdb_data_no_dups.csv")
df2 = pd.read_csv("pdb_data_seq.csv")

In [3]:
# === Keep only the required columns & rename ===
df1_filtered = df1[["structureId", "classification", "macromoleculeType", "residueCount", "resolution"]]
df1_filtered = df1_filtered.rename(columns={
    "residueCount": "residueCount_df1",
    "macromoleculeType": "macromoleculeType_df1"
})

df2_filtered = df2[["structureId", "sequence", "residueCount"]]
df2_filtered = df2_filtered.rename(columns={
    "residueCount": "residueCount_df2"
})

In [4]:
# === Merge on structureId ===
merged_df = pd.merge(df1_filtered, df2_filtered, on="structureId", how="inner")

# === Drop duplicate sequences ===
merged_df = merged_df.drop_duplicates(subset="sequence")

# Drop rows where any column has a missing value
merged_df = merged_df.dropna()

# === Save the final filtered dataset ===
merged_df.to_csv("filtered_dataset.csv", index=False)

print(f"Filtered dataset saved as filtered_dataset.csv with {len(merged_df)} rows.")

Filtered dataset saved as filtered_dataset.csv with 154 rows.
